# 데이터 전처리와 분석

## 학습 목표
- 정규화(Normalization)의 개념과 필요성 이해
- 데이터 전처리 기법 학습 (중복 제거, 타입 변환, 결측치 처리)
- 라벨 인코딩과 원-핫 인코딩 이해
- 데이터 분석 기초 (기술통계, 조건부 필터링)
- 파일 입출력 처리

---

## 1. 정규화(Normalization)의 개념

### 1.1 선형 회귀의 기본 개념
- **목표**: `y = ax + b`에서 자동으로 `a`와 `b`를 찾아내는 과정
- **과정**:
  ```
  a, b = 2, 1
  x = [1, 2, 3, 4, 5, 6, 7, ...]
  y = [...] (기대값)
  real_y = [...] (실제값)
  오차의 제곱의 합: 4843783873
  
  a, b = 3, 1 (최적화 과정)
  ```

### 1.2 다중 선형 회귀
- **수식**: `y = w1*x1 + w2*x2 + w3*x3 + w4*x4 + ... + b`
- **구성 요소**:
  - `y`: 예측하려는 목표값
  - `x1, x2, ...`: y에 영향을 미치는 입력 변수들
  - `w1, w2, ...`: 기울기들 (가중치)
  - `b`: 절편

### 1.3 정규화의 필요성
- **문제점**: 모든 데이터(필드, 특성, 피처)들의 단위가 다를 수 있음
- **해결책**: 단위를 맞추는 과정이 **정규화(Normalization)**
- **적용 분야**: 
  - 서포트 벡터 머신(SVM)
  - 머신러닝 알고리즘
  - 딥러닝 모델

> **중요**: 머신러닝과 딥러닝에서는 반드시 단위가 비슷해야 합니다!

In [92]:
# 1.1 자동차 연비 데이터 로드 및 기본 정보 확인
import pandas as pd 

data = pd.read_csv('./data/auto-mpg.csv')
print("=== 데이터 기본 정보 ===")
print(data.info())
print("\n=== 데이터 상위 5행 ===")
print(data.head())

=== 데이터 기본 정보 ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    object 
 3   horsepower    396 non-null    float64
 4   weight        398 non-null    int64  
 5   acceleration  398 non-null    float64
 6   model-year    398 non-null    int64  
dtypes: float64(3), int64(3), object(1)
memory usage: 21.9+ KB
None

=== 데이터 상위 5행 ===
    mpg  cylinders displacement  horsepower  weight  acceleration  model-year
0  18.0          8            ?       130.0    3504          12.0          70
1  15.0          8          350       165.0    3693          11.5          70
2  18.0          8          318       150.0    3436          11.0          70
3  16.0          8          304       150.0    3433          12.0          70
4  17.0       

In [93]:
# 1.2 컬럼명 변경 - 프로그램 안에서 직접 변경
data.columns = ['mpg', 'cyl', 'disp', 'power', 'weight', 'acce', 'model']
print("=== 컬럼명 변경 후 ===")
print(data.head())

=== 컬럼명 변경 후 ===
    mpg  cyl disp  power  weight  acce  model
0  18.0    8    ?  130.0    3504  12.0     70
1  15.0    8  350  165.0    3693  11.5     70
2  18.0    8  318  150.0    3436  11.0     70
3  16.0    8  304  150.0    3433  12.0     70
4  17.0    8  302  140.0    3449  10.5     70


In [94]:
# 1.3 정규화 및 단위 환산 적용
# Min-Max 정규화: (값 - 최소값) / (최대값 - 최소값)
data['mpg2'] = (data['mpg'] - data['mpg'].min()) / (data['mpg'].max() - data['mpg'].min())

# 단위환산: MPG(Miles Per Gallon) → KPL(Kilometers Per Liter)
mpg_unit = 1.60934 / 3.78541  # 1마일 = 1.60934km, 1갤런 = 3.78541리터
data['kpl'] = (data['mpg'] * mpg_unit).round(2)  # 소수점 이하 2자리 반올림

print("=== 정규화 및 단위환산 결과 ===")
print(data.head())
print(f"\n정규화 범위: {data['mpg2'].min():.3f} ~ {data['mpg2'].max():.3f}") 

=== 정규화 및 단위환산 결과 ===
    mpg  cyl disp  power  weight  acce  model      mpg2   kpl
0  18.0    8    ?  130.0    3504  12.0     70  0.239362  7.65
1  15.0    8  350  165.0    3693  11.5     70  0.159574  6.38
2  18.0    8  318  150.0    3436  11.0     70  0.239362  7.65
3  16.0    8  304  150.0    3433  12.0     70  0.186170  6.80
4  17.0    8  302  140.0    3449  10.5     70  0.212766  7.23

정규화 범위: 0.000 ~ 1.000


## 2. 중복 데이터 제거 (Duplicate Removal)

### 2.1 중복 제거의 필요성
- **문제점**: 동일한 데이터가 여러 번 존재할 경우 분석 결과 왜곡
- **해결책**: `duplicated()`, `drop_duplicates()` 함수 활용
- **옵션**: 
  - `subset`: 특정 컬럼 기준으로 중복 확인
  - `keep`: 중복 데이터 중 어떤 것을 유지할지 결정

In [95]:
# 2.1 중복 데이터 예시 생성
import pandas as pd 

# 승객 코드별 목적지와 가격 정보 (중복 데이터 포함)
data = {
    'passenger_code': ['A101', 'A102', 'A103', 'A101', 'A104', 'A101', 'A103'],
    'target': ['광주', '서울', '부산', '광주', '대구', '광주', '부산'],
    'price': [25000, 27000, 45000, 25000, 35000, 27000, 45000]
}

df = pd.DataFrame(data)
print("=== 원본 데이터 (중복 포함) ===")
print(df)
print(f"\n전체 데이터 개수: {len(df)}개")

=== 원본 데이터 (중복 포함) ===
  passenger_code target  price
0           A101     광주  25000
1           A102     서울  27000
2           A103     부산  45000
3           A101     광주  25000
4           A104     대구  35000
5           A101     광주  27000
6           A103     부산  45000

전체 데이터 개수: 7개


In [96]:
# 2.2 중복 데이터 확인
print("=== 중복 데이터 확인 ===")
col = df['passenger_code'].duplicated()  # True 또는 False의 리스트로 반환
print("passenger_code 중복 여부:")
print(col)
print(f"\n중복된 데이터 개수: {col.sum()}개")
print(f"고유한 승객 코드 개수: {df['passenger_code'].nunique()}개")

=== 중복 데이터 확인 ===
passenger_code 중복 여부:
0    False
1    False
2    False
3     True
4    False
5     True
6     True
Name: passenger_code, dtype: bool

중복된 데이터 개수: 3개
고유한 승객 코드 개수: 4개


In [97]:
# 2.3 전체 필드 기준 중복 제거
print("=== 전체 필드 기준 중복 제거 ===")
df2 = df.drop_duplicates()  # 모든 필드가 완전히 일치하는 데이터만 삭제
print(df2)
print(f"\n중복 제거 후 데이터 개수: {len(df2)}개 (제거된 데이터: {len(df) - len(df2)}개)")

=== 전체 필드 기준 중복 제거 ===
  passenger_code target  price
0           A101     광주  25000
1           A102     서울  27000
2           A103     부산  45000
4           A104     대구  35000
5           A101     광주  27000

중복 제거 후 데이터 개수: 5개 (제거된 데이터: 2개)


In [98]:

# 2.4 특정 컬럼 기준 중복 제거 (subset 사용)
print("=== 승객 코드 기준 중복 제거 ===")
df3 = df.drop_duplicates(subset=['passenger_code'])
print(df3)
print(f"\n승객 코드 기준 중복 제거 후: {len(df3)}개 (제거된 데이터: {len(df) - len(df3)}개)")

=== 승객 코드 기준 중복 제거 ===
  passenger_code target  price
0           A101     광주  25000
1           A102     서울  27000
2           A103     부산  45000
4           A104     대구  35000

승객 코드 기준 중복 제거 후: 4개 (제거된 데이터: 3개)


In [99]:
# 2.5 다중 컬럼 기준 중복 제거
print("=== 승객 코드 + 목적지 기준 중복 제거 ===")
df4 = df.drop_duplicates(subset=['passenger_code', 'target'])
print(df4)
print(f"\n승객 코드 + 목적지 기준 중복 제거 후: {len(df4)}개 (제거된 데이터: {len(df) - len(df4)}개)")

=== 승객 코드 + 목적지 기준 중복 제거 ===
  passenger_code target  price
0           A101     광주  25000
1           A102     서울  27000
2           A103     부산  45000
4           A104     대구  35000

승객 코드 + 목적지 기준 중복 제거 후: 4개 (제거된 데이터: 3개)


## 3. 데이터 타입 변환 (Type Conversion)

### 3.1 타입 변환의 필요성
- **문제점**: 데이터 타입이 맞지 않으면 올바른 분석이 어려움
- **해결책**: `astype()` 함수를 사용하여 적절한 타입으로 변환
- **주요 타입**:
  - `object`: 문자열 타입
  - `int64`, `float64`: 수치 타입
  - `category`: 범주형 타입
  - `datetime`: 날짜/시간 타입

### 3.2 파이썬 버전별 차이점
- **최신 버전**: 문자열 데이터라도 수치 형태면 자동으로 수치 자료로 처리
- **이전 버전**: 수동으로 타입 변환이 필요할 수 있음

In [100]:
# 3.1 자동차 데이터 다시 로드 및 타입 확인
import pandas as pd 
import numpy as np 

data = pd.read_csv('./data/auto-mpg.csv')
print("=== 원본 데이터 타입 정보 ===")
print(data.info())
print("\n=== 원본 데이터 샘플 ===")
print(data.head())

=== 원본 데이터 타입 정보 ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    object 
 3   horsepower    396 non-null    float64
 4   weight        398 non-null    int64  
 5   acceleration  398 non-null    float64
 6   model-year    398 non-null    int64  
dtypes: float64(3), int64(3), object(1)
memory usage: 21.9+ KB
None

=== 원본 데이터 샘플 ===
    mpg  cylinders displacement  horsepower  weight  acceleration  model-year
0  18.0          8            ?       130.0    3504          12.0          70
1  15.0          8          350       165.0    3693          11.5          70
2  18.0          8          318       150.0    3436          11.0          70
3  16.0          8          304       150.0    3433          12.0          70
4  17.0    

## 4. 라벨 인코딩과 원-핫 인코딩 (Label & One-Hot Encoding)

### 4.1 머신러닝을 위한 데이터 전처리와 수치화

머신러닝 및 딥러닝 모델은 **수치 데이터**만을 입력으로 사용할 수 있습니다.  
따라서 모든 입력 데이터는 반드시 수치형으로 변환되어야 하며, 문자열이나 범주형 데이터는 적절한 전처리 과정을 거쳐야 합니다.

### 4.2 문자열 데이터의 처리

#### 카테고리화가 가능한 문자열
- 예: 연비 등급이 'A', 'B', 'C', 'D'와 같이 정해진 범주로만 이루어진 경우
- 이를 **카테고리 타입**으로 변환할 수 있습니다
- 허용된 범주(A, B, C, D) 이외의 값은 데이터에 포함되지 않도록 차단해야 합니다
- 카테고리화가 불가능한 문자열(예: 이름, 고유 식별자 등)은 머신러닝에 불필요하므로 **삭제**해야 합니다

### 4.3 카테고리 데이터의 수치화 방법

#### (1) 라벨 인코딩(Label Encoding)
- 각 카테고리 값을 정수로 매핑합니다
- 예시:
  ```
  A → 1, B → 2, C → 3, D → 4
  ```
- **주의**: 카테고리 간에 순서나 크기 의미가 없을 때는 부적합할 수 있습니다

#### (2) 원-핫 인코딩(One-Hot Encoding)
- 각 카테고리를 별도의 컬럼으로 만들어, 해당 카테고리에만 1, 나머지는 0으로 표시
- 예시:
  ```
  원본: A, B, C, D
  변환 결과:
    A  B  C  D
    1  0  0  0   (A)
    0  1  0  0   (B)
    0  0  1  0   (C)
    0  0  0  1   (D)
  ```
- **장점**: 카테고리 간의 순서/크기 정보가 없을 때 적합
- **단점**: 카테고리 개수가 많으면(예: 단어 10만 개) 컬럼 수가 매우 커질 수 있음

### 4.4 데이터 전처리의 일반 원칙

1. **수치화가 가능한 데이터**는 반드시 수치형으로 변환
2. **카테고리화가 불가능한 문자열**(예: 이름, 고유번호 등)은 삭제
3. **결측치/이상치 처리**: NaN, 잘못된 값 등은 적절히 처리
4. **정규화/표준화**: 수치형 데이터의 단위가 다를 경우, 정규화 또는 표준화 필요

### 4.5 실무 적용 예시

- **연비 등급**(문자열): 'A', 'B', 'C', 'D' → 원-핫 인코딩 또는 라벨 인코딩
- **이름**(문자열): 삭제
- **자동차 모델**(카테고리): 카테고리 타입으로 변환
- **연비, 배기량** 등(수치): 결측치 처리 후 float 타입으로 변환, 필요시 정규화

### 4.6 대용량 텍스트 데이터의 원-핫 인코딩

- 단어가 100,000개라면, 각 샘플은 1개의 1과 99,999개의 0으로 이루어진 벡터가 됨
- 차원이 너무 커질 경우, **임베딩(embedding)** 등 다른 방법을 고려해야 함

> **결론**: 머신러닝을 위해서는 모든 입력 데이터가 수치형이어야 하며, 이를 위해 문자열/카테고리형 데이터는 적절한 인코딩 및 전처리 과정을 거쳐야 합니다.

In [101]:
# 3.2 컬럼명 변경 및 데이터 타입 확인
# 타입이 맞지 않을 경우 전환을 해서 사용해야 한다
# 현재 사용하는 파이썬 버전은 문자열 데이터라도 수치 형태면 자동으로 수치자료로 처리한다
# 파이썬 버전에 따라 다르게 동작할 수도 있다
data.columns = ['mpg', 'cyl', 'disp', 'power', 'weight', 'acce', 'model']
print("=== 컬럼명 변경 후 데이터 타입 ===")
print(data.dtypes)
print("\n=== 변경된 데이터 샘플 ===")
print(data.head())

=== 컬럼명 변경 후 데이터 타입 ===
mpg       float64
cyl         int64
disp       object
power     float64
weight      int64
acce      float64
model       int64
dtype: object

=== 변경된 데이터 샘플 ===
    mpg  cyl disp  power  weight  acce  model
0  18.0    8    ?  130.0    3504  12.0     70
1  15.0    8  350  165.0    3693  11.5     70
2  18.0    8  318  150.0    3436  11.0     70
3  16.0    8  304  150.0    3433  12.0     70
4  17.0    8  302  140.0    3449  10.5     70


In [102]:
# 3.3 displacement 컬럼의 이상값 확인 및 처리
print("=== displacement 컬럼의 고유값 확인 ===")
print(data['disp'].unique())  # 중복된 것을 배제하고 고유값만 표시

print(f"\n고유값 개수: {data['disp'].nunique()}개")
print(f"'?' 값이 포함된 행 개수: {(data['disp'] == '?').sum()}개")

# ✅ 올바른 방법: 잘못된 데이터를 NaN으로 변경
data = data.replace({'disp': {'?': np.nan}})

print("\n=== '?' 값을 NaN으로 변경 후 ===")
print(data.head())
print(f"\ndisplacement 컬럼의 결측치 개수: {data['disp'].isnull().sum()}개")

=== displacement 컬럼의 고유값 확인 ===
['?' '350' '318' '304' '302' '429' '454' '440' '455' '390' '383' '340'
 '400' '113' '198' '199' '200' '97' '110' '107' '104' '121' '360' '307'
 '140' '98' '232' '225' '250' '351' '258' '122' '116' '79' '88' '71' '72'
 '91' '97.5' '70' '120' '96' '108' '155' '68' '114' '156' '76' '83' '90'
 '231' '262' '134' '119' '171' '115' '101' '305' '85' '130' '168' '111'
 '260' '151' '146' '80' '78' '105' '131' '163' '89' '267' '86' '183' '141'
 '173' '135' '81' '100' '145' '112' '181' '144']

고유값 개수: 83개
'?' 값이 포함된 행 개수: 1개

=== '?' 값을 NaN으로 변경 후 ===
    mpg  cyl disp  power  weight  acce  model
0  18.0    8  NaN  130.0    3504  12.0     70
1  15.0    8  350  165.0    3693  11.5     70
2  18.0    8  318  150.0    3436  11.0     70
3  16.0    8  304  150.0    3433  12.0     70
4  17.0    8  302  140.0    3449  10.5     70

displacement 컬럼의 결측치 개수: 1개


In [105]:
# 범주형으로 바꾼다 
print(data.dtypes)
data['model'] = data['model'].astype('category')
print(data.dtypes)
# 3.5 model 컬럼을 범주형(Category)으로 변환
print("=== 범주형 변환 전 데이터 타입 ===")
print(data.dtypes)

print(f"\nmodel 컬럼의 고유값 개수: {data['model'].nunique()}개")
print(f"model 컬럼의 고유값: {sorted(data['model'].unique())}")

# model 컬럼을 범주형으로 변환
data['model'] = data['model'].astype('category')

print("\n=== 범주형 변환 후 데이터 타입 ===")
print(data.dtypes)

mpg        float64
cyl          int64
disp        object
power      float64
weight       int64
acce       float64
model     category
dtype: object
mpg        float64
cyl          int64
disp        object
power      float64
weight       int64
acce       float64
model     category
dtype: object
=== 범주형 변환 전 데이터 타입 ===
mpg        float64
cyl          int64
disp        object
power      float64
weight       int64
acce       float64
model     category
dtype: object

model 컬럼의 고유값 개수: 13개
model 컬럼의 고유값: [70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82]

=== 범주형 변환 후 데이터 타입 ===
mpg        float64
cyl          int64
disp        object
power      float64
weight       int64
acce       float64
model     category
dtype: object


In [106]:
print(f"\n범주형 변환 후 메모리 사용량 비교:")
print(f"- 범주형으로 변환하면 메모리 효율성이 증가합니다")
print(f"- 특히 중복되는 값이 많을 때 효과적입니다")


범주형 변환 후 메모리 사용량 비교:
- 범주형으로 변환하면 메모리 효율성이 증가합니다
- 특히 중복되는 값이 많을 때 효과적입니다


DataFrame의 버그( 카테고리)- 그 범위를 벗어나는 데이터는 받으면 안된다.<br>
카테고리타입에 데이터 추가했더니 스스로 float로 바뀌었다.<br>
허용되면 안된다.

```python 
data = data.append({'mpg':90, 'model':93}, ignore_index=True)
data.loc[len(data)] = {'mpg':90, 'model':93}

```
모델타입이 카테코리라서 없는 카테고리 값을 추가할 경우에 오류가 발생한다.<br>
파일을 읽을때 범위가 결정되어서 93이 해당사항이 없어서 에러가 발생한다.<br>
반드시 범주형으로 되어야 할것들은 범주형으로 바꿔줘야 한다.<br>

In [ ]:
print(data.dtypes)
print( data['model'].value_counts())# 빈도표, 범주형의 경우에는 엄청 중요하다 

mpg        float64
cyl          int64
disp       float64
power      float64
weight       int64
acce       float64
model     category
dtype: object
model
73    40
78    36
76    34
82    31
75    30
80    29
79    29
81    29
70    28
71    28
72    28
77    28
74    27
Name: count, dtype: int64


# 분석

In [ ]:
import pandas as pd 

df = pd.read_csv("./data/auto-mpg.csv")
# print(df)

print(df.head())
print(df.tail())

    mpg  cylinders displacement  horsepower  weight  acceleration  model-year
0  18.0          8            ?       130.0    3504          12.0          70
1  15.0          8          350       165.0    3693          11.5          70
2  18.0          8          318       150.0    3436          11.0          70
3  16.0          8          304       150.0    3433          12.0          70
4  17.0          8          302       140.0    3449          10.5          70
      mpg  cylinders displacement  horsepower  weight  acceleration  \
393  27.0          4          140        86.0    2790          15.6   
394  44.0          4           97        52.0    2130          24.6   
395  32.0          4          135        84.0    2295          11.6   
396  28.0          4          120        79.0    2625          18.6   
397  31.0          4          119        82.0    2720          19.4   

     model-year  
393          82  
394          82  
395          82  
396          82  
397          82

In [ ]:
row, col = df.shape 
print("데이터전체개수 ", row, col)

# info ->데이터 정보를 요약해서 보여준다 
print( df.info()) 

데이터전체개수  398 7
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    object 
 3   horsepower    396 non-null    float64
 4   weight        398 non-null    int64  
 5   acceleration  398 non-null    float64
 6   model-year    398 non-null    int64  
dtypes: float64(3), int64(3), object(1)
memory usage: 21.9+ KB
None


In [ ]:
# 통계량 - 기초통계학 중요 : 평균, 개수, 표준편차(std),최소값,최대값, 중간값(2/4), 1/4분위수
print(df.describe())

              mpg   cylinders  horsepower       weight  acceleration  \
count  398.000000  398.000000  396.000000   398.000000    398.000000   
mean    23.514573    5.454774  104.189394  2970.424623     15.568090   
std      7.815984    1.701004   38.402030   846.841774      2.757689   
min      9.000000    3.000000   46.000000  1613.000000      8.000000   
25%     17.500000    4.000000   75.000000  2223.750000     13.825000   
50%     23.000000    4.000000   92.000000  2803.500000     15.500000   
75%     29.000000    8.000000  125.000000  3608.000000     17.175000   
max     46.600000    8.000000  230.000000  5140.000000     24.800000   

       model-year  
count  398.000000  
mean    76.010050  
std      3.697627  
min     70.000000  
25%     73.000000  
50%     76.000000  
75%     79.000000  
max     82.000000  


In [ ]:
# df2 = df[조건식] 조건식이 True 데이터 셋만 가져온다 
df2 = df[ df["cylinders"]==8]
print(df.shape)
print(df2.shape)
# value_counts() - 카테고리형 데이터  2 4 8 
print(df2["cylinders"].value_counts())

# 실린더-카테고리타입 , 평균이나 표준값 따위가 의미가 없는 데이터 타입 
# value_counts() - 분할표 
print(df["cylinders"].value_counts())

(398, 7)
(103, 7)
cylinders
8    103
Name: count, dtype: int64
cylinders
4    204
8    103
6     84
3      4
5      3
Name: count, dtype: int64


In [ ]:
# 모델발표연도가 70이고 연비가 25이상(and, or-python연산자는 사용못해)
import numpy as np   
df2 = df[ np.logical_and( df["model-year"]==70 , df["mpg"]>=25) ] 
print(df2)

# 모델발표연도가 70이고 연비가 25이상(and, or-python연산자는 사용못해) 데이터 건수 
print(df2.shape[0])

# 모델과 모델별 제품 개수 
print(df["model-year"].value_counts())


# 문제1. 80년대에 나온 제품들을 모델과 개수로 나타내기 
print(df[df['model-year']//10==8].value_counts())
print(df[np.logical_and(df['model-year']>=80, df['model-year']<=89)].value_counts())


# 문제2. 73, 78, 76년에 나온 제품만 모두 출력하기 
"""
df2 = df[ (df['model-year']==73) | (df['model-year']==78) | (df['model-year']==76) ]
print(df2)
df2 = df[ (df['model-year']>=80) & (df['model-year']<=89) & (df['mpg']>=25) ]
df[(df['model-year'].isin(range(80, 90))) & (df['mpg'] >= 25)]
"""

# 문제3. 80년대에 출시한 제품중에서 연비가 25 이상인 제품의 정보만 출력하기 
print(df[np.logical_and(df['model-year']//10==8, df.mpg>=25)])

# 문제4. 70년대에 출시된 모델중에서 실린더가 4개인 제품의 정보만 출력하기
  
# 문제5. 80년대에 출시된 모델들을 실린더 개수와 제품개수로 출력하기 

# | or 비트연산자   & and 연산자
# df2 = df[ (df['model-year']==73) | (df['model-year']==78) | (df['model-year']==76) ]


a = 5 & 7  #  0101 & 0111 = 0101 
print(a)

     mpg  cylinders displacement  horsepower  weight  acceleration  model-year
18  27.0          4           97        88.0    2130          14.5          70
19  26.0          4           97        46.0    1835          20.5          70
20  25.0          4          110        87.0    2672          17.5          70
22  25.0          4          104        95.0    2375          17.5          70
23  26.0          4          121       113.0    2234          12.5          70
5
model-year
73    40
78    36
76    34
82    31
75    30
80    29
79    29
81    29
70    29
71    28
72    28
77    28
74    27
Name: count, dtype: int64
mpg   cylinders  displacement  horsepower  weight  acceleration  model-year
17.6  6          225           85.0        3465    16.6          81            1
19.1  6          225           90.0        3381    18.7          80            1
20.2  6          200           88.0        3060    17.1          81            1
22.0  6          232           112.0       2835    

In [ ]:
import pandas as pd 

# 분석에서 데이터가 크게 두종류가 있다 - 분석방법이 다르다 
# 연속형데이터, 평균값이 중요하다 연비 40, 30, 25, 회귀분석, 회귀(Regressior)    
# 불연속형데이터, 범주형, 카테고리, 평균값이 중요하지 않다. 빈도수가 중요하다. 
# 발생빈도수 - 실린더수 3 5 4 6 8 , 분류분석, 로지스틱회귀, 분류(Classifier)

# header가 3번째 줄에 있음 
data = pd.read_csv("./data/auto-mpg.csv")

# value_counts 각 데이터별 고유카운트-빈도수, 발생빈도수를 카운트 한다  
print( data['model-year'].value_counts())

# 평균, 최대, 최소 
print("연비평균 : ", data['mpg'].mean())
print("연비최대 : ", data['mpg'].max())
print("연비최소 : ", data['mpg'].min())
print("연비중간 : ", data['mpg'].median())
print("연비분산 : ", data['mpg'].var())
print("연비표준편차 : ", data['mpg'].std())

print("1사분위수 : ", data['mpg'].quantile(0.25))
print("2사분위수 : ", data['mpg'].quantile(0.5))
print("3사분위수 : ", data['mpg'].quantile(0.75))

model-year
73    40
78    36
76    34
82    31
75    30
80    29
79    29
81    29
70    29
71    28
72    28
77    28
74    27
Name: count, dtype: int64
연비평균 :  23.514572864321607
연비최대 :  46.6
연비최소 :  9.0
연비중간 :  23.0
연비분산 :  61.089610774274405
연비표준편차 :  7.815984312565782
1사분위수 :  17.5
2사분위수 :  23.0
3사분위수 :  29.0


# 결측치

In [ ]:
import numpy as np 
import pandas as pd 


s1 = pd.Series([1,2,3,4,np.nan]) #프로그램에서 nan값을 직접 입력할때는 np.nan을 사용해야 한다 
print(s1.isnull()) 
print(s1.isnull().sum()) #[ False, False, False, False, True]
#c언어 True는 1이고 False는 0으로 해석  
#[0,0,0,0,1].sum() 

data = pd.read_csv("./data/data.csv")
print(data.shape) 
print(data.info())
print(data.describe())

print( data['height'].value_counts()) 
print( data['height'].value_counts(dropna=False)) 

print( data['height'].isnull().sum()) 
print( data['weight'].isnull().sum()) 

#결측치가 발생하면 행이나 열을 삭제시키거나 평균값이나 중간값으로 대체를 한다. 
#표준편차가 크면 평균값 보다는 중간값으로 대체를 하는것이 좋다 

data2 = data.dropna( how='any', axis=0) #행중에 NaN값 있으면 행 전체를 삭제해라 
print(data2.shape)#25, 3으로 준다 

data2 = data.dropna( how='any', axis=1) #열중에 NaN값이 하나라도 있으면 삭제한다. 
print(data2.shape)#30, 1으로 준다 name열 빼고 전체가 삭제된다. 

#thresh 라는 옵션 - 최소한의 실효성있는 데이터 개수를 유지해라
data2 = data.dropna(thresh=28, axis=1) #데이터개수가 27개인 열은 삭제된다. 
print(data2.shape)
print(data2.head())

data = pd.read_csv("./data/auto-mpg.csv")
print( data.info() )

print(data['horsepower'].isnull().sum())
data = data.dropna(how='any', axis=0)
print(data.shape)

0    False
1    False
2    False
3    False
4     True
dtype: bool
1
(30, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   name    30 non-null     object 
 1   height  27 non-null     float64
 2   weight  28 non-null     float64
dtypes: float64(2), object(1)
memory usage: 852.0+ bytes
None
           height      weight
count   27.000000   28.000000
mean   177.888889   67.107143
std     12.096832   25.725530
min    167.000000   12.000000
25%    172.000000   59.000000
50%    175.000000   65.000000
75%    179.000000   77.250000
max    230.000000  160.000000
height
175.0    3
172.0    3
174.0    2
176.0    2
177.0    2
179.0    2
173.0    2
169.0    2
180.0    1
168.0    1
181.0    1
189.0    1
193.0    1
167.0    1
230.0    1
171.0    1
187.0    1
Name: count, dtype: int64
height
172.0    3
175.0    3
NaN      3
176.0    2
174.0    2
177.0    2
179.0    2


In [ ]:
# 5.2 결측치 대체 방법 (평균값 사용)
import pandas as pd 

# 데이터 로드
data = pd.read_csv("./data/data.csv")

print("=== 원본 데이터 정보 ===")
print(data.head())
print(data.info())

print("\n=== 결측치 현황 ===")
print("height 결측치:", data["height"].isnull().sum())
print("weight 결측치:", data["weight"].isnull().sum())

# 데이터프레임 전체의 결측치를 확인하기
print("\n전체 컬럼별 결측치:")
print(data.isnull().sum())

# 평균값 계산
mean_height = data['height'].mean() 
mean_weight = data['weight'].mean() 

print(f"\n평균 키: {mean_height:.2f}cm")
print(f"평균 몸무게: {mean_weight:.2f}kg")

# ✅ 올바른 결측치 처리 방법
data.fillna({'height': mean_height, 'weight': mean_weight}, inplace=True)

print("\n=== 누락데이터 교체 후 ===")
print(data.isnull().sum())
print("\n처리 완료된 데이터:")
print(data.head())

=== 원본 데이터 정보 ===
  name  height  weight
0   A1   180.0    92.0
1   A2   176.0    70.0
2   A3   175.0    65.0
3   A4   172.0    64.0
4   A5   168.0     NaN
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   name    30 non-null     object 
 1   height  27 non-null     float64
 2   weight  28 non-null     float64
dtypes: float64(2), object(1)
memory usage: 852.0+ bytes
None

=== 결측치 현황 ===
height 결측치: 3
weight 결측치: 2

전체 컬럼별 결측치:
name      0
height    3
weight    2
dtype: int64

평균 키: 177.89cm
평균 몸무게: 67.11kg

=== 누락데이터 교체 후 ===
name      0
height    0
weight    0
dtype: int64

처리 완료된 데이터:
  name  height     weight
0   A1   180.0  92.000000
1   A2   176.0  70.000000
2   A3   175.0  65.000000
3   A4   172.0  64.000000
4   A5   168.0  67.107143


## 6. 파일 입출력 처리 (File I/O Operations)

### 6.1 다양한 파일 형식 지원
- **CSV 파일**: `pd.read_csv()`, `df.to_csv()`
- **Excel 파일**: `pd.read_excel()`, `df.to_excel()`
- **JSON 파일**: `pd.read_json()`, `df.to_json()`
- **기타**: Parquet, HDF5, SQL 등

### 6.2 파일 읽기 옵션
- `header`: 헤더 행 지정
- `index_col`: 인덱스로 사용할 컬럼 지정
- `encoding`: 인코딩 방식 지정
- `sep`: 구분자 지정

In [ ]:
# 6.1 기본 CSV 파일 읽기 및 데이터 처리
import pandas as pd 

df = pd.read_csv("./data/score.csv") 
# 데이터프레임으로 만들면 자동으로 행 인덱스가 부여된다
print("=== 원본 성적 데이터 ===")
print(df.head()) 
print(f"\n컬럼명: {list(df.columns)}")
print(f"인덱스: {df.index}")
print(f"데이터 형태: {df.shape}")

# 새로운 컬럼 추가 (총점과 평균)
df["total"] = df.kor + df.eng + df.mat  # 키값에 하이픈이 들어갈 경우 이렇게 쓸 수 없다
df["avg"] = df.total // 3               # 키 추가시에는 df["키"] 형태로 써야 한다  

print("\n=== 총점과 평균 추가 후 ===")
print(df)

=== 원본 성적 데이터 ===
  name  kor  eng  mat
0  홍길동   90   90   90
1  임꺽정   80   80   80
2  장길산   70   70   70
3  홍경래   90   80   70
4  이징옥   60   50   50

컬럼명: ['name', 'kor', 'eng', 'mat']
인덱스: RangeIndex(start=0, stop=5, step=1)
데이터 형태: (5, 4)

=== 총점과 평균 추가 후 ===
  name  kor  eng  mat  total  avg
0  홍길동   90   90   90    270   90
1  임꺽정   80   80   80    240   80
2  장길산   70   70   70    210   70
3  홍경래   90   80   70    240   80
4  이징옥   60   50   50    160   53


In [ ]:
# 6.2 다양한 파일 형식 처리 및 저장 옵션
import pandas as pd 

# 헤더가 특정 행에 있는 CSV 파일 읽기
df = pd.read_csv("./data/score_header.csv", header=3) 
print("=== 헤더 지정 CSV 읽기 ===")
print(df.head()) 
print(f"컬럼명: {list(df.columns)}")
print(f"인덱스: {df.index}")

# 데이터 처리
df["total"] = df.kor + df.eng + df.mat  # 키 값에 하이픈이 들어갈 경우 이렇게 쓸 수 없다
df["avg"] = df.total // 3               # 키 추가 시에는 df["키"] 형태로 써야 한다  

print("\n=== 처리된 데이터 ===")
print(df)

# 다양한 저장 옵션
print("\n=== 파일 저장 ===")
df.to_csv("./data/score_result1.csv", mode="w")                                    # 기본 저장 (인덱스 포함)
df.to_csv("./data/score_result2.csv", mode="w", index=False, encoding='utf-8-sig') # 인덱스 제외, UTF-8 BOM 인코딩
print("CSV 파일 저장 완료")

# Excel 파일 읽기
print("\n=== Excel 파일 읽기 ===")
try:
    df_excel = pd.read_excel("./data/score.xlsx")
    print(df_excel)
    print("Excel 파일 읽기 성공")
except FileNotFoundError:
    print("Excel 파일을 찾을 수 없습니다.")
except Exception as e:
    print(f"Excel 파일 읽기 오류: {e}")

=== 헤더 지정 CSV 읽기 ===
  name  kor  eng  mat
0  홍길동   90   90   90
1  임꺽정   80   80   80
2  장길산   70   70   70
3  홍경래   90   80   70
4  이징옥   60   50   50
컬럼명: ['name', 'kor', 'eng', 'mat']
인덱스: RangeIndex(start=0, stop=5, step=1)

=== 처리된 데이터 ===
  name  kor  eng  mat  total  avg
0  홍길동   90   90   90    270   90
1  임꺽정   80   80   80    240   80
2  장길산   70   70   70    210   70
3  홍경래   90   80   70    240   80
4  이징옥   60   50   50    160   53

=== 파일 저장 ===
CSV 파일 저장 완료

=== Excel 파일 읽기 ===
  name  kor  eng  mat
0  홍길동   90   90   90
1  임꺽정   80   80   80
2  장길산   70   70   70
3  홍경래   90   80   70
4  이징옥   60   50   50
5  일지매   90   90    0
Excel 파일 읽기 성공


## 7. 학습 요약 및 정리

### 7.1 오늘 학습한 주요 내용

#### 1️⃣ 정규화 (Normalization)
- **Min-Max 정규화**: `(값 - 최소값) / (최대값 - 최소값)`
- **목적**: 서로 다른 단위의 데이터를 0~1 범위로 통일
- **중요성**: 머신러닝/딥러닝에서 필수적인 전처리 과정

#### 2️⃣ 중복 데이터 제거
- `duplicated()`: 중복 여부 확인
- `drop_duplicates()`: 중복 데이터 제거
- **옵션**: `subset` (특정 컬럼 기준), `keep` (유지할 데이터 선택)

#### 3️⃣ 데이터 타입 변환
- `astype()`: 데이터 타입 변환
- **주요 타입**: `object`, `int64`, `float64`, `category`, `datetime`
- **범주형 데이터**: 카테고리 타입으로 변환하여 메모리 효율성 증대

#### 4️⃣ 인코딩 기법
- **라벨 인코딩**: 카테고리를 정수로 매핑 (순서 의미 있을 때)
- **원-핫 인코딩**: 카테고리별 독립적인 컬럼 생성 (순서 의미 없을 때)
- **적용 원칙**: 머신러닝은 수치 데이터만 처리 가능

#### 5️⃣ 결측치 처리
- **확인**: `isnull()`, `info()`, `describe()`
- **제거**: `dropna()` (행/열 단위)
- **대체**: `fillna()` (평균값, 중간값, 최빈값 등)
- **권장**: 표준편차가 클 때는 중간값 사용

#### 6️⃣ 데이터 분석 기초
- **연속형 데이터**: 평균, 표준편차, 분위수 등 → 회귀 분석
- **범주형 데이터**: 빈도수, 분할표 등 → 분류 분석
- **조건부 필터링**: `logical_and()`, `logical_or()`, 비트 연산자

#### 7️⃣ 파일 입출력
- **읽기**: `read_csv()`, `read_excel()`, `read_json()`
- **저장**: `to_csv()`, `to_excel()`, `to_json()`
- **옵션**: `header`, `index`, `encoding`, `mode`

### 7.2 핵심 포인트

✅ **데이터 전처리는 머신러닝의 80%**  
✅ **정규화는 서로 다른 단위의 데이터를 통일하는 필수 과정**  
✅ **결측치와 이상치는 반드시 처리해야 함**  
✅ **범주형 데이터는 적절한 인코딩으로 수치화**  
✅ **파일 형식에 따른 적절한 읽기/쓰기 옵션 활용**

### 7.3 다음 단계

- 고급 전처리 기법 (스케일링, 표준화)
- 특성 엔지니어링 (Feature Engineering)
- 데이터 시각화 (Matplotlib, Seaborn)
- 머신러닝 모델 적용 준비

---

> **💡 실무 팁**: 데이터 분석의 성공은 좋은 전처리에서 시작됩니다. 항상 데이터의 품질을 먼저 확인하고, 적절한 전처리를 통해 깨끗한 데이터를 만들어야 합니다!
